In [2]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [4]:
%mkdir ch16
!touch ch16/__init__.py

mkdir: cannot create directory ‘ch16’: File exists


## 16.1 生成器如何进化为协程

- 协程的底层架构在[“PEP 342—Coroutines via EnhancedGenerators”](https://www.python.org/dev/peps/pep-0342/)中定义，并在Python 2.5（2006 年）实现
  - `yield` 关键字可以在表达式中使用，作为右值使用
  - 生成器 API 增加了 `.send(value)` 方法
    - 生成器的调用者可以使用 `.send(..)` 方法发送数据
    - 发送的数据会成为生成器函数中 `yield` 表达式的值
  - 添加了 `.throw(..)` 方法, 让调用方抛出异常，在生成器中处理
  - 添加了 `.close()` 方法，作用是终止生成器
- 有了上述的几个特性，使得生成器可以作为协程使用
  - 协程是指一个过程，这个过程与调用方协作，产出由调用方提供的值

- 协程最近的演进来自 Python 3.3（2012 年）实现的 [“PEP 380—Syntax for Delegating to a Subgenerator”](https://www.python.org/dev/peps/pep-0380/)。 
- PEP 380 对生成器函数的句法做了两处改动，以便更好地作为协程使用
  - 生成器可以返回一个值
  - 新引入了 yield from 句法，使用它可以把复杂的生成器重构成小型的嵌套生成器，省去了之前把生成器的工作委托给子生成器所需的大量样板代码

## 16.2 用作协程的生成器的基本行为

###### 示例 16-1　可能是协程最简单的使用演示

In [ ]:
def simple_coroutine():  # 定义体中含有 yield 关键字，即构成协程
  print('-> coroutine started')
  x = yield
  print('-> coroutine received:', x)

In [ ]:
my_coro = simple_coroutine()
my_coro  # 调用函数得到生成器对象

<generator object simple_coroutine at 0x7f089a312a98>

In [ ]:
next(my_coro)  # 首先调用 next 函数，让生成器启动至 yield 语句处暂停

-> coroutine started


In [ ]:
my_coro.send(42)  # 调用 send 方法后，会将 x 赋值为 42，然后恢复协程的运行
          # 运行至函数末尾时，会跟一般的生成器一样，抛出 StopIteration 异常

-> coroutine received: 42


StopIteration: ignored

###### 查看协程的状态

- 当前状态可使用 `inspect.getgeneratorstate(...)` 函数确定，该函数会返回下述字符串中的一个
  - `GEN_CREATED`
    - 等待开始执行
  - `GEN_RUNNING` 
    - 生成器正在执行
    - 只有在多线程应用中才能看到这个状态
  - `GEN_SUSPENDED`
    - 在 `yield` 表达式处暂停
  - `GEN_CLOSED` 
    - 执行结束

###### 预激(prime)协程

- 如果协程还没激活（即，状态是 `GEN_CREATED`）
  - 需要调用 `next(my_coro)` 激活协程
  - 也可以调用 `my_coro.send(None)`，效果一样
  - 目的是让协程向前执行到第一个 `yield` 表达式，准备好作为活跃的协程使用

- 如果创建协程对象后立即把 `None` 之外的值发给它，会出现错误

In [ ]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: ignored

###### 示例 16-2 产出两个值的协程

In [ ]:
def simple_coro2(a):
  print('-> Started: a =', a)
  b = yield a
  print('-> Received: b = ', b)
  c = yield a + b
  print('-> Received: c = ', c)

In [ ]:
my_coro2 = simple_coro2(14)

In [ ]:
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [ ]:
next(my_coro2)

-> Started: a = 14


14

In [ ]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [ ]:
my_coro2.send(28)

-> Received: b =  28


42

In [ ]:
my_coro2.send(99)

-> Received: c =  99


StopIteration: ignored

In [ ]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

- `simple_coro2` 执行的三个阶段
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200614164817.png width=800>

## 16.3 示例：使用协程计算移动平均值

###### 示例 16-3　`coroaverager0.py`：定义一个计算移动平均值的协程

In [ ]:
%%writefile ch16/coroaverager0.py
def averager():
  total = 0.0
  count = 0
  average = None
  while True:
    term = yield average
    total += term
    count += 1
    average = total / count

Writing ch16/coroaverager0.py


In [ ]:
from ch16.coroaverager0 import averager

In [ ]:
coro_avg = averager()
next(coro_avg) # 预激协程，由于相当于发送的值为 None, 所以不会输出值
coro_avg.send(10)

10.0

In [ ]:
coro_avg.send(30)

20.0

In [ ]:
coro_avg.send(5)

15.0

## 16.4 预激协程的装饰器

- 使用 `yield from` 句法调用协程时，会自动预激，因此与此装饰器不兼容

###### 示例 16-5　coroutil.py：预激协程的装饰器

In [ ]:
%%writefile ch16/coroutil.py
from functools import wraps

def coroutine(func):
  """
  装饰器：向前执行到第一个 yield 表达式，预激 func
  """
  @wraps(func)
  def primer(*args, **kwargs):
    gen = func(*args, **kwargs)
    next(gen)  # 预激生成器
    return gen  # 返回生成器
  return primer

Writing ch16/coroutil.py


###### 示例 16-6　coroaverager1.py：使用示例 16-5 中定义的@coroutine 装饰器定义并测试计算移动平均值的协程

In [ ]:
%%writefile ch16/coroaverager1.py
from ch16.coroutil import coroutine

@coroutine
def averager():
  total = 0.0
  count = 0
  average = None
  while True:
    term = yield average
    total += term
    count += 1
    average = total / count

Overwriting ch16/coroaverager1.py


In [ ]:
from ch16.coroaverager1 import averager

In [ ]:
coro_avg = averager()
from inspect import getgeneratorstate
getgeneratorstate(coro_avg)  # 此状态表明协程已准备好，可以接收值

'GEN_SUSPENDED'

In [ ]:
coro_avg.send(10)

10.0

In [ ]:
coro_avg.send(30)

20.0

In [ ]:
coro_avg.send(5)

15.0

## 16.5 终止协程和异常处理

- 客户代码可以在生成器对象上调用 `throw` 和 `close` 方法，显式地把异常发给协程
- `generator.throw(exc_type[, exc_value[, traceback]])`
  - 致使生成器在暂停的 `yield` 表达式处抛出指定的异常
  - 如果生成器处理了抛出的异常，代码会向前执行到下一个 `yield` 表达式，而产出的值会成为调用 `generator.throw` 方法得到的返回值
  - 如果生成器没有处理抛出的异常，异常会向上冒泡，传到调用方的上下文中
- `generator.close()`
  - 致使生成器在暂停的 `yield` 表达式处抛出 `GeneratorExit` 异常
  - 如果生成器没有处理这个异常，或者抛出了 `StopIteration` 异常（通常是指运行到结尾），调用方不会报错
  - 如果收到 `GeneratorExit` 异常，生成器一定不能产出值，否则解释器会抛出 `RuntimeError` 异常
  - 生成器抛出的其他异常会向上冒泡，传给调用方。

###### 示例 16-7　未处理的异常会导致协程终止

In [ ]:
from ch16.coroaverager1 import averager

In [ ]:
coro_avg = averager()

In [ ]:
coro_avg.send(40)

40.0

In [ ]:
coro_avg.send(50)

45.0

In [ ]:
coro_avg.send('spam')

TypeError: ignored

In [ ]:
coro_avg.send(60)  # 协程内部没有处理异常，协程会终止。如果试图重新激活协程，会抛出 StopIteration 异常

StopIteration: ignored

###### 示例 16-8 coro_exc_demo.py： 学习在协程中处理异常的测试代码

In [ ]:
%%writefile ch16/coro_exec_demo.py
class DemoException(Exception):
  """为这次演示定义的异常类型"""


def demo_exc_handling():
  print('-> coroutine started')
  while True:
    try:
      x = yield
    except DemoException:  # 特别处理 DemoException 异常
      print('*** DemoException handled. Continuing...')
    else:  # 如果没有异常，显示接收到的值
      print('-> coroutine received: {!r}'.format(x))
  raise RuntimeError('This line should never run.')  # 此行永远不会执行

Writing ch16/coro_exec_demo.py


###### 示例 16-9　激活和关闭 `demo_exc_handling`，没有异常

In [ ]:
from ch16.coro_exec_demo import demo_exc_handling
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [ ]:
exc_coro.send(11)

-> coroutine received: 11


In [ ]:
exc_coro.send(22)

-> coroutine received: 22


In [ ]:
exc_coro.close()

In [ ]:
from inspect import getgeneratorstate
getgeneratorstate(exc_coro)

'GEN_CLOSED'

###### 示例 16-10　把 `DemoException` 异常传入 `demo_exc_handling` 不会导致协程中止

In [ ]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [ ]:
exc_coro.send(11)

-> coroutine received: 11


In [ ]:
from ch16.coro_exec_demo import DemoException
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [ ]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

###### 示例 16-11　如果无法处理传入的异常，协程会终止

In [ ]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [ ]:
exc_coro.send(11)

-> coroutine received: 11


In [ ]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: ignored

In [ ]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

###### 示例 16-12 `coro_finally_demo.py`: 使用 `try/finally` 块在协程终止时执行操作

In [ ]:
%%writefile ch16/coro_finally_demo.py
class DemoException(Exception):
  """为这次演示定义的异常类型"""


def demo_finally():
  print('-> coroutine started')
  try:
    while True:
      try:
        x = yield
      except DemoException:
        print('*** DemoException handled. Continuing...')
      else:
        print('-> coroutine received: {!r}'.format(x))
  finally:
    print('-> coroutine ending')

Writing ch16/coro_finally_demo.py


In [ ]:
from ch16.coro_finally_demo import demo_finally

In [ ]:
exc_coro = demo_finally()
next(exc_coro)

-> coroutine started


In [ ]:
exc_coro.send(11)

-> coroutine received: 11


In [ ]:
exc_coro.throw(ZeroDivisionError)

-> coroutine ending


ZeroDivisionError: ignored

## 16.6 让协程返回值

- `yield` 返回值时会产生 `StopIteration` 异常，返回的值会储存在异常的 `value` 属性中
- 对于 `yield from` 结构来说，其会在内部自动捕获  `StopIteration` 异常，并将 `value` 属性的值变为 `yield from` 表达式的值

###### 示例 16-13　coroaverager2.py：定义一个求平均值的协程，让它返回一个结果

In [ ]:
%%writefile ch16/coroaverager2.py
from collections import namedtuple

Result = namedtuple('Result', 'count averaage')

def averager():
  total = 0.0
  count = 0
  average = None
  while True:
    term = yield
    if term is None:
      break  # 只有协程正常退出，才能返回值
    total += term
    count += 1
    average = total / count
  return Result(count ,average)

Writing ch16/coroaverager2.py


###### 示例 16-14 coroaverager2.py: 说明 `averager` 行为的 doctest

In [ ]:
from ch16.coroaverager2 import averager
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)  # 抛出的 StopIteration 异常对象的 value 属性

StopIteration: ignored

###### 示例 16-15 捕获 StopIteration 异常，获取 averager 返回值

In [ ]:
from ch16.coroaverager2 import averager
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
  coro_avg.send(None)
except StopIteration as exc:
  result = exc.value

result

Result(count=3, averaage=15.5)

## 16.7 使用 `yield from`

- `yield from` 是全新的语言结构，类似于其它语言的 `await` 关键字
  - 在生成器 `gen` 中使用 `yield from subgen()` 时， `subgen` 会获得控制权，把产出值传给 `gen` 的调用方，即调用方可以直接控制 `subgen` 
  - 与此同时， `gen` 会阻塞，等待 `subgen` 终止
- `yield from x` 表达式对 `x` 对象所做的第一件事是调用 `iter(x)`，从中获取迭代器，因此 `x` 可以是任何可迭代对象
- `yield from` 的主要功能是打开双向通道,把最外层的调用方与最内层的子生成器连接起来
  - 这样二者可以直接发送和产出值,还可以直接传入异常,而不用在位于中间的协程中添加大量处理异常的样板代码
- 与 `yield from` 相关的专门术语
  - 委派生成器
    - 包含 `yield from <iterable>` 表达式的生成器函数
  - 子生成器
    - 从 `yield from` 表达式 `<iterable>` 部分获取的生成器
  - 调用方
    - 指代委派生成器的客户端代码
- `yield from` 用法的示意图
  - ![用法示意图](https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200616154549.png)
  - 委派生成器在 yield from 表达式处暂停时,调用方可以直接把数据发给子生成器,子生成器再把产出的值发给调用方
  - 子生成器返回之后,解释器会抛出 `StopIteration` 异常,并把返回值附加到异常对象上,此时委派生成器会恢复

###### 示例 16-16 使用 `yield from` 链接可迭代的对象

In [ ]:
def chain(*iterables):
  for it in iterables:
    yield from it

In [ ]:
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

###### 示例 16-17 coroaverager3.py: 使用 `yield from` 计算平均值并输出报告

In [61]:
%%writefile ch16/coroaverager3.py
from collections import namedtuple
from inspect import getgeneratorstate
from coroutil import coroutine

Result = namedtuple('Result', 'count average')


# 子生成器
def averager():
  total = 0
  count = 0
  average = None
  while True:
    term = yield  # main 中发送的各个值绑定到 term 变量上
    if term is None: # 终止条件
      break
    total += term
    count += 1
    average = total / count
  return Result(count, average)


# 委派生成器
def grouper(results, key):
  while True:  # 每次迭代都会创建一个 averager 实例：每个实例都是作为协程使用的生成器对象
    results[key] = yield from averager()


# 客户端代码，即调用方
def main(data):
  results = {}
  for key, values in data.items():
    group = grouper(results, key)
    next(group)  # 预激 group 协程，激活的是委派生成器，子生成器会由 yield from 自动预激
    for value in values:  
      group.send(value)  # 把各个 value 传给 grouper。传入的值最终到达 averager 函数中 term = yield 那一行;grouper 永远不知道传入的值是什么
    group.send(None)  #! 重要  把 None 传入 grouper,导致当前的 averager 实例终止,也让 grouper 继续运行,再创建一个 averager 实例,处理下一组值
  report(results)


# 输出报告
def report(results):
  for key, result in sorted(results.items()):
    group, unit = key.split(';')
    print('{:2} {:5} averaging {:.2f}{}'.format(
        result.count, group, result.average, unit
    ))


data = {
  'girls;kg':
  [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
  'girls;m':
  [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
  'boys;kg':
  [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
  'boys;m':
  [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}


if __name__ == '__main__':
  main(data)

Overwriting ch16/coroaverager3.py


In [62]:
!python ch16/coroaverager3.py

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


## 16.8 `yield from` 的意义

- `yield from` 的行为有以下四点
  - 子生成器产出的值都直接传给委派生成器的调用方（即客户端代码）
  - 使用 `send()` 方法发给委派生成器的值都直接传给子生成器
    - 如果传送的是 `None`， 那么会调用子生成器的 `__next__()` 方法
    - 如果发送的不是 `None`，则会调用子生成器的 `send()` 方法
    - 如果调用的方法抛出 `StopIteration` 异常，那么委派生成器会恢复运行。任何其他异常都会向上冒泡，传给委派生成器
  - 生成器退出时，生成器(或子生成器)中的 `return expr` 表达式会触发 `StopIteration(expr)` 异常抛出
  - `yield from` 表达式的值是子生成器终止时传给 `StopIteration` 异常的第一个参数
- `yield from` 结构的另外两个特性与异常和终止有关
  - 传入委派生成器的异常，除了 `GeneratorExit` 之外都传给子生成器的 `throw()` 方法
    - 如果调用 `throw()` 方法时抛出 `StopIteration` 异常，委派生成器恢复运行
    - `StopIteration` 之外的异常会向上冒泡，传给委派生成器
  - 如果把 `GeneratorExit` 异常传入委派生成器，或者在委派生成器上调用 `close()` 方法时
    - 会在子生成器上调用 `close()` 方法，如果它有的话；如果没有，则不进行任何操作
    - 如果调用 `close()` 方法导致异常抛出，则异常会向上冒泡，传给委派生成器；否则，委派生成器抛出 `GeneratorExit` 异常
  - `yield from` 会预激子生成器 

###### 示例 16-19　伪代码，等效于委派生成器中的 `RESULT = yield from EXPR` 语句

```python
_i = iter(EXPR)  # EXPR 可以是任何可迭代对象
try:
  _y = next(_i)  # 预激子生成器，结果保留在 _y 中，作为产出的第一个值
except StopIteration as _e:
  _r = _e.value  # 如果产生 StopIteration，则获取相应的返回值
else:
  while 1:  # 此循环运行时，委派生成器会阻塞，只作为调用方和子生成器之间的通道
    try:
      _s = yield _y  # 产出子生成器当前产出的值，等待调用方发送 _s 中保存的值，此代码清单中，只有这一个 yield 表达式
    except GeneratorExit as _e:  # 关闭委派生成器和子生成器，由于子生成器可以是任何可迭代对象，因此其可能没有 close 方法
      try:
        _m = _i.close
      except AttributeError:
        pass
      else:
        _m()
      raise _e  # 向上抛出异常
    except BaseException as _e:  # 处理 .throw() 方法传入的异常
      _x = sys.exc_info()
      try:
        _m = _i.throw
      except AttributeError:
        raise _e
      else:  # 如果子生成器有 throw 方法，调用它并传入调用方发来的异常
        try:
          _y = _m(*_x)
        except StopIteration as _e:
          _r = _e.value  # 如果抛出 StopIteration 异常，则从中获取结果，赋值给 _r，循环结束，委派生成器恢复运行
          break
    else:  # 如果产出值时没有异常
      try: 
        if _s is None:  # 如果调用方发送的值是 None, 则调用 next 方法，否则调用 send 方法
          _y = next(_i)
        else:
          _y = _i.send(_s)
      except StopIteration as _e: # 如果抛出 StopIteration 异常，则从中获取结果，赋值给 _r，循环结束，委派生成器恢复运行
        _r = _e.value 
        break
  
RESULT = _r
  
  ```

## 16.9 使用案例：使用协程做离散事件仿真

### 16.9.1 离散事件仿真简介

- 回合制游戏是离散仿真的例子
  - 游戏的状态只在玩家操作时变化，而且一旦玩家决定下一步怎么走了，仿真钟就会冻结
- `SimPy` 是一个实现离散事件仿真的 Python 包
- 可以通过一个协程表示离散事件仿真系统中的各个进程

### 16.9.2 出租车队运营仿真

###### 示例 16-20 `taxi_sim.py`: `taxi_process` 协程，实现各辆出租车的活动

In [ ]:
import collections


Event = collections.namedtuple('Event', 'time proc action')


def taxi_process(ident, trips, start_time=0):
  """
  每次改变状态时创建事件，把控制权让给仿真器
  """
  time = yield Event(start_time, ident, 'leave garage')  
  for i in range(trips):
    time = yield Event(time, ident, 'pick up passenger')  # 产生 Event 实例，表示拉到乘客。协程会在这里暂停，等待主循环发送时间
    time = yield Event(time, ident, 'drop off passenger')  # 产生 Event 实例，表示乘客下车。协程会在这里暂停，等待主循环发送时间

  yield Event(time, ident, 'going home')
  # 出租车进程结束


###### 示例 16-21 驱动 `taxi_process` 协程

In [ ]:
taxi = taxi_process(ident=13, trips=2, start_time=0)

In [ ]:
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [ ]:
taxi.send(_.time + 7)  # 在控制台中， _ 变量绑定的是前一个结果

Event(time=7, proc=13, action='pick up passenger')

In [ ]:
taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [ ]:
taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [ ]:
taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [ ]:
taxi.send(_.time + 1)

Event(time=84, proc=13, action='going home')

In [ ]:
taxi.send(_.time + 10)

StopIteration: ignored

###### 示例 16-23 Simulator，一个简单的离散事件仿真类；关重的重点是 `run` 方法

In [ ]:
class Simulator:

  def __init__(self, procs_map):
    self.events = queue.PriorityQueue()  # 保存排定事件的 PriorityQueue 对象（会按对象的第一个属性值进行排序），此例中会按时间排序
    self.procs = dict(procs_map)  # 避免在仿真的过程中，修改传入的参数

  def run(self, end_time):
    """
    排定并显示事件，直到时间结束
    """
    # 排定出租车的第一个事件
    for _, proc in sorted(self.procs.items()):  # 获取按键排序的元素
      first_event = next(proc)  # 预激协程，并接收产生的第一个 Event 对象
      self.events.put(first_event)  # 将各个对像添回到 PriorityQueue 对象中

    # 仿真系统的主循环
    sim_time = 0
    while sim_time < end_time:
      if self.events.empty():
        print('*** end of events ***')
        break

      current_event = self.events.get()  # 获取优先队列中 time 属性最小的 Event 对象
      sim_time, proc_id, previous_action = current_event
      print('taxi:', proc_id, proc_id * ' ', current_event)
      active_proc = self.procs[proc_id]  # 从字典中获取当前活动的出租车的协程
      next_time = sim_time + compute_duration(previous_action)  # 传入前一个动作，将结果加到 sim_time 上，计算出下一次活动的时间间隔
      try:
        next_event = active_proc.send(next_time)
      except StopIteration:
        del self.procs[proc_id]  # 从字典中删除相应的协程
      else:
        self.events.put(next_event)  # 否则将 next_event 放入队列中
    else:  # 如果循环由于仿真时间到了而退出，显示待完成的事件数量
      msg = '*** end of simulation time: {} events pending ***'
      print(msg.format(self.events.qsize()))


###### 出租车离散事件仿真整体代码

In [3]:
%%writefile ch16/taxi_sim.py
"""
出租车仿真程序
"""

import random
import collections
import queue
import argparse
import time

DEFAULT_NUMBER_OF_TAXIS = 3 
DEFAULT_END_TIME = 180
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPATURE_INTERVAL = 5

Event = collections.namedtuple('Event', 'time proc action')


# BEGIN TAXI_PROCESS
def taxi_process(ident, trips, start_time=0):
  """
  每次改变状态时创建事件，把控制权让给仿真器

  通过 time 将多个事件串联起来
  """
  time = yield Event(start_time, ident, 'leave garage')  
  for i in range(trips):
    time = yield Event(time, ident, 'pick up passenger')  # 产生 Event 实例，表示拉到乘客。协程会在这里暂停，等待主循环发送时间
    time = yield Event(time, ident, 'drop off passenger')  # 产生 Event 实例，表示乘客下车。协程会在这里暂停，等待主循环发送时间

  yield Event(time, ident, 'going home')
  # 出租车进程结束
# END TAXI_PROCESS


# BEGIN TAXI_SIMULATOR
class Simulator:

  def __init__(self, procs_map):
    self.events = queue.PriorityQueue()  # 保存排定事件的 PriorityQueue 对象（会按对象的第一个属性值进行排序），按时间正向排序
    self.procs = dict(procs_map)  # 避免在仿真的过程中，修改传入的参数

  def run(self, end_time):
    """
    排定并显示事件，直到时间结束
    """
    # 排定出租车的第一个事件
    for _, proc in sorted(self.procs.items()):  # 获取按键排序的元素
      first_event = next(proc)  # 预激协程，并接收产生的第一个 Event 对象
      self.events.put(first_event)  # 将各个对像添回到 PriorityQueue 对象中

    # 仿真系统的主循环
    sim_time = 0
    while sim_time < end_time:
      if self.events.empty():
        print('*** end of events ***')
        break

      current_event = self.events.get()  # 获取优先队列中 time 属性最小的 Event 对象
      sim_time, proc_id, previous_action = current_event
      print('taxi:', proc_id, proc_id * '\t', current_event)
      active_proc = self.procs[proc_id]  # 从字典中获取当前活动的出租车的协程
      next_time = sim_time + compute_duration(previous_action)  # 传入前一个动作，将结果加到 sim_time 上，计算出下一次活动的时间
      try:
        next_event = active_proc.send(next_time)
      except StopIteration:
        del self.procs[proc_id]  # 从字典中删除相应的协程
      else:
        self.events.put(next_event)  # 否则将 next_event 放入队列中
    else:  # 如果循环由于仿真时间到了而退出，显示待完成的事件数量
      msg = '*** end of simulation time: {} events pending ***'
      print(msg.format(self.events.qsize()))
# END TAXI_SIMULATOR


def compute_duration(previous_action):
    """
    使用指数分布计算操作的耗时
    """
    if previous_action in ['leave garage', 'drop off passenger']:
        # 新状态是四处排佪
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        # 新状态是行程开始
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return int(random.expovariate(1/interval)) + 1
        

def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS, seed=None):
    """
    初始化随机生成器，构建过程，运行仿真程序
    """
    if seed is not None:
        random.seed(seed)  # 获取可复现的结果

    taxis = {i: taxi_process(i, (i+1)*2, i*DEPATURE_INTERVAL) for i in range(num_taxis)}
    sim = Simulator(taxis)
    sim.run(end_time)

    
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Taxi fleet simulator')
    parser.add_argument('-e', '--end-time', type=int, 
                        default=DEFAULT_END_TIME,
                        help='simulation end time; default =%s' % DEFAULT_END_TIME)
    parser.add_argument('-t', '--taxis', type=int, 
                        default=DEFAULT_NUMBER_OF_TAXIS, 
                        help='number of taxis running; defalut=%s' % DEFAULT_NUMBER_OF_TAXIS)
    parser.add_argument('-s', '--seed', type=int, default=None,
                        help='random generator seed (for testing)')
    
    args = parser.parse_args()
    main(args.end_time, args.taxis, args.seed)

Overwriting ch16/taxi_sim.py


In [4]:
!python3 ch16/taxi_sim.py --help

usage: taxi_sim.py [-h] [-e END_TIME] [-t TAXIS] [-s SEED]

Taxi fleet simulator

optional arguments:
  -h, --help            show this help message and exit
  -e END_TIME, --end-time END_TIME
                        simulation end time; default =180
  -t TAXIS, --taxis TAXIS
                        number of taxis running; defalut=3
  -s SEED, --seed SEED  random generator seed (for testing)


In [5]:
!python3 ch16/taxi_sim.py -s 3 -e 120

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=2, proc=0, action='pick up passenger')
taxi: 1 	 Event(time=5, proc=1, action='leave garage')
taxi: 1 	 Event(time=8, proc=1, action='pick up passenger')
taxi: 2 		 Event(time=10, proc=2, action='leave garage')
taxi: 2 		 Event(time=15, proc=2, action='pick up passenger')
taxi: 2 		 Event(time=17, proc=2, action='drop off passenger')
taxi: 0  Event(time=18, proc=0, action='drop off passenger')
taxi: 2 		 Event(time=18, proc=2, action='pick up passenger')
taxi: 2 		 Event(time=25, proc=2, action='drop off passenger')
taxi: 1 	 Event(time=27, proc=1, action='drop off passenger')
taxi: 2 		 Event(time=27, proc=2, action='pick up passenger')
taxi: 0  Event(time=28, proc=0, action='pick up passenger')
taxi: 2 		 Event(time=40, proc=2, action='drop off passenger')
taxi: 2 		 Event(time=44, proc=2, action='pick up passenger')
taxi: 1 	 Event(time=55, proc=1, action='pick up passenger')
taxi: 1 	 Event(time=59, proc=1, a